# Сегментация тетрадей

## Detectron2 baseline

В данном ноутбуке представлен baseline модели сегментации текста в школьных тетрадях с помощью фреймворка detectron2. Вы можете (и это даже лучше) использовать другие модели (например UNET, mmdet), или написать полностью свою.

# 0. Установка библиотек

Установка библиотек, под которым запускается данный бейзлайн.

In [ ]:
!nvidia-smi

In [ ]:
# !pip install gdown
# !gdown https://drive.google.com/uc?id=1VOojDMJe7RAxryQ2QKXrqA7CvhsnzJ_z

In [ ]:
# !rm -rf __MACOSX
# !rm -rf data

In [ ]:
# %%capture
# !unzip -u /home/jovyan/nto_final_data.zip
# !mv data/train_segmentation data/train

In [ ]:
# !pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install git+https://github.com/facebookresearch/detectron2.git

In [ ]:
# !pip install opencv-pyth
# !pip install tensorflow==2.1.0

In [ ]:
# %%capture
# !pip install albumentations==0.4.6
# !pip install pycocotools
# !pip install imutils
# !git clone https://github.com/MarkPotanin/copy_paste_aug_detectron2.git

# !cp ./copy_paste_aug_detectron2/coco.py ./
# !cp ./copy_paste_aug_detectron2/copy_paste.py ./
# !cp ./copy_paste_aug_detectron2/visualize.py ./

## 1. Загрузить необходимые библиотеки для создания и обучения модели

In [ ]:
import cv2
import random
import json
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import shutil

In [ ]:
import torch, torchvision
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog,DatasetCatalog
from detectron2.data.datasets import register_coco_instances, load_coco_json
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer
from detectron2.engine import HookBase
from detectron2.evaluation.evaluator import DatasetEvaluator
import pycocotools.mask as mask_util

from matplotlib import pyplot as plt
from tqdm import tqdm
import numpy as np
import gc

import logging
logger = logging.getLogger('detectron2')
logger.setLevel(logging.CRITICAL)

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator
import detectron2.data.transforms as T
from detectron2.data import build_detection_train_loader, build_detection_test_loader
import copy
import json
from detectron2.evaluation import inference_on_dataset

In [ ]:
def clear_cache():
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()

Прежде чем переходить к загрузке данных посмотрим, доступны ли нам GPU-мощности. 

In [ ]:
print('GPU: ' + str(torch.cuda.is_available()))

# 2. Валидационный датасет

Для валидации наших моделей нам неплохо было создать из обучающих данных валидационный датасет. Для этого разделим наш датасет на две части - для обучения и для валидации. Для этого просто создадим два новых файлика с аннотациями, куда раздельно запишем исиходную информацию об аннотациях.

In [ ]:
#Подгрузим аннотации train
with open('data/train/annotations.json') as f:
    annotations = json.load(f)

In [ ]:
len(annotations['images'])

In [ ]:
#Пустой словарь для аннотаций валидации
annotations_val = {}
#Список категорий такой же как в train
annotations_val['categories'] = annotations['categories']

#Пустой словарь для аннотаций нового train
annotations_train = {}
#Список категорий такой же как в train
annotations_train['categories'] = annotations['categories']

#Положим в валидацию каждое 10 изображение из исходного train, а остальные - в новый train
annotations_val['images'] = []
annotations_train['images'] = []
for num, img in enumerate(annotations['images']):
    if num % 110 == 0:
        annotations_val['images'].append(img)
    else:
        annotations_train['images'].append(img)

#Положим в список аннотаций валидации только те аннотации, которые относятся к изображениям из валидации. 
#А в список аннотаций нового train - только те, которые относятся к нему
val_img_id = [i['id'] for i in annotations_val['images']]
train_img_id = [i['id'] for i in annotations_train['images']]

annotations_val['annotations'] = []
annotations_train['annotations'] = []

for annot in annotations['annotations']:
    if annot['image_id'] in val_img_id:
        annotations_val['annotations'].append(annot)
    elif annot['image_id'] in train_img_id:
        annotations_train['annotations'].append(annot)
    else:
        print('Аннотации нет ни в одном наборе')

In [ ]:
for i, element in enumerate(annotations_train["images"]):
    if element["file_name"] == "41_3.JPG":
        print(element["id"])
        del annotations_train["images"][i]

for i, element in enumerate(annotations_train["annotations"]):
    if element["image_id"] == 405:
        print("Done")
        del annotations_train["annotations"][i]

In [ ]:
for i, element in enumerate(annotations_val["images"]):
    if element["file_name"] == "41_3.JPG":
        print(element["id"])
        del annotations_val["images"][i]

for i, element in enumerate(annotations_val["annotations"]):
    if element["image_id"] == 405:
        print("Done")
        del annotations_val["annotations"][i]

In [ ]:
try: os.remove("data/train/images/41_3.JPG")
except: pass

In [ ]:
clear_cache()

Готово! Аннотации для валидации и новой обучающей выборки готовы, теперь просто сохраним их в формате json, и положим в папке. Назовем аннотации annotations_new.json, чтобы новая набор аннотаций для train (без множества val) не перезаписал исходные аннотации.

In [ ]:
if not os.path.exists('data/val'):
    os.makedirs('data/val')
if not os.path.exists('data/val/images'):
    os.makedirs('data/val/images')

Скопируем изображения, которые относятся к валидации, в папку val/images

In [ ]:
for i in annotations_val['images']:
    shutil.copy('data/train/images/'+i['file_name'],'data/val/images/')

Запишем новые файлы с аннотациями для train и val.

In [ ]:
with open('data/val/annotations_new.json', 'w') as outfile:
    json.dump(annotations_val, outfile)


with open('data/train/annotations_new.json', 'w') as outfile:
    json.dump(annotations_train, outfile)

# 3. Регистрация датасета

Зарегистрируем выборки в detectron2 для дальнейшей подачи на обучение модели.

In [ ]:
for d in ['train','val']:
    DatasetCatalog.register("my_dataset_"+d, lambda d=d: load_coco_json("./data/{}/annotations_new.json".format(d),
    image_root= "./data/train/images",\
    dataset_name="my_dataset_"+d,extra_annotation_keys=['bbox_mode']))

После регистрации можно загружать выборки, чтобы иметь возможность посмотреть на них глазами. Первой загрузим обучающую выборку в **dataset_dicts_train**

In [ ]:
dataset_dicts_train = DatasetCatalog.get("my_dataset_train")
train_metadata = MetadataCatalog.get("my_dataset_train")

И тестовую выборку в **dataset_dicts_val**

In [ ]:
dataset_dicts_val = DatasetCatalog.get("my_dataset_val")
val_metadata = MetadataCatalog.get("my_dataset_val")

Посмотрим на размер получившихся выборок - эта операция в python осуществляется при помощи функции **len()**

In [ ]:
print('Размер обучающей выборки (Картинки): {}'.format(len(dataset_dicts_train)))
print('Размер тестовой выборки (Картинки): {}'.format(len(dataset_dicts_val)))

Итак, у нас в распоряжении 588 изображения для тренировки, и 66 - для проверки качества.

**Посмотрим на размеченные фотографии из валидации**

In [ ]:
import os
from IPython.display import Image
@interact
def show_images(file=range(len(dataset_dicts_val))):
    example = dataset_dicts_val[file]
    image = utils.read_image(example["file_name"], format="RGB")
    plt.figure(figsize=(3,3),dpi=200)
    visualizer = Visualizer(image[:, :, ::-1], metadata=val_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(example)
    plt.imshow(vis.get_image()[:, :,::-1])
    plt.show()

In [ ]:
dataset_dicts_val[0]["file_name"]

# copy-paste

In [ ]:
# import scripts
# https://github.com/MarkPotanin/copy_paste_aug_detectron2/blob/main/detectron2_copypaste.ipynb
from copy_paste import CopyPaste
from coco import CocoDetectionCP
import albumentations as A
from pycocotools import mask
from skimage import measure

In [ ]:
aug_list = [A.Resize(1960, 1960),\
            A.OneOf([A.HorizontalFlip(),
                     A.RandomRotate90()], p=0.75),
            A.OneOf([A.Blur(),
                     A.MotionBlur(),
                     A.GaussNoise(),
                     A.ImageCompression(quality_lower=75)],p=0.5),
        CopyPaste(blend=True, sigma=1, pct_objects_paste=0.3, p=1.0) #pct_objects_paste is a guess
    ]

transform = A.Compose(
            aug_list, bbox_params=A.BboxParams(format="coco")
        )

In [ ]:
data = CocoDetectionCP(
    './data/train/images', 
    './data/train/annotations_new.json', 
    transform
)

In [ ]:
data_id_to_num = {i:q for q,i in enumerate(data.ids)}

ALL_IDS = list(data_id_to_num.keys())

dataset_dicts_train = [i for i in dataset_dicts_train if i['image_id'] in ALL_IDS]

BOX_MODE = dataset_dicts_train[0]['annotations'][0]['bbox_mode']

In [ ]:
class MyMapper:
    """Mapper which uses `detectron2.data.transforms` augmentations"""
    def __init__(self, cfg, is_train: bool = True):

        self.is_train = is_train

        mode = "training" if is_train else "inference"
        #print(f"[MyDatasetMapper] Augmentations used in {mode}: {self.augmentations}")

    def __call__(self, dataset_dict):
        dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
        img_id = dataset_dict['image_id']
        
        
        aug_sample = data[data_id_to_num[img_id]]
        
        image = aug_sample['image']
        
        image =  cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))
        
        
        bboxes = aug_sample['bboxes']
        box_classes = np.array([b[-2] for b in bboxes])
        boxes = np.stack([b[:4] for b in bboxes], axis=0)
        mask_indices = np.array([b[-1] for b in bboxes])
        
        
        masks = aug_sample['masks']
        
        annos = []
        
        for enum,index in enumerate(mask_indices):
            curr_mask = masks[index]
            
            fortran_ground_truth_binary_mask = np.asfortranarray(curr_mask)
            encoded_ground_truth = mask.encode(fortran_ground_truth_binary_mask)
            ground_truth_area = mask.area(encoded_ground_truth)
            ground_truth_bounding_box = mask.toBbox(encoded_ground_truth)
            contours = measure.find_contours(curr_mask, 0.5)
            
            annotation = {
        "segmentation": [],
        "iscrowd": 0,
        "bbox": ground_truth_bounding_box.tolist(), 
        "category_id": train_metadata.thing_dataset_id_to_contiguous_id[box_classes[enum]]  ,
        "bbox_mode":BOX_MODE
                
                
    }
            for contour in contours:
                contour = np.flip(contour, axis=1)
                segmentation = contour.ravel().tolist()
                annotation["segmentation"].append(segmentation)
                
            annos.append(annotation)
        

        image_shape = image.shape[:2]  # h, w

        
        instances = utils.annotations_to_instances(annos, image_shape)
        dataset_dict["instances"] = utils.filter_empty_instances(instances)
        
        clear_cache()
        return dataset_dict

#   4 Обучение модели

**4.1. Определяем конфигурацию**

Прежде чем начать работать с самой моделью, нам нужно определить ее параметры и спецификацию обучения

Создаем конфигурацию и загружаем архитектуру модели с предобученными весами (на COCO - датасете, содержащем $80$ популярных категорий объектов и более $300000$ изображений) для распознавания объектов.

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")) 
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "/home/jovyan/model_final (1).pth"

В целом, вы можете посмотреть и другие архитектуры в зоопарке [моделей](https://github.com/facebookresearch/detectron2/blob/master/MODEL_ZOO.md).

Теперь задаем параметры самой модели и обучения модели

In [ ]:
height, width = 10000, 10000
for element in annotations_train["images"]:
    height = min(height, element["height"])
    width = min(width, element["width"])
print(height, width)

In [ ]:
# Загружаем названия обучающией и тестовой выборок в настройки
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)

cfg.TEST.EVAL_PERIOD = 20

# cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 64

# Часто имеет смысл сделать изображения чуть меньшего размера, чтобы 
# обучение происходило быстрее. Поэтому мы можем указать размер, до которого будем изменяться наименьшая 
# и наибольшая из сторон исходного изображения.
cfg.INPUT.MIN_SIZE_TRAIN = 1960 
cfg.INPUT.MAX_SIZE_TRAIN = 1960

cfg.INPUT.MIN_SIZE_TEST = cfg.INPUT.MIN_SIZE_TRAIN
cfg.INPUT.MAX_SIZE_TEST = cfg.INPUT.MAX_SIZE_TRAIN

# Также мы должны сказать модели ниже какой вероятности определения она игнорирует результат. 
# То есть, если она найдет на картинке еду, но вероятность правильного определения ниже 0.5, 
# то она не будет нам сообщать, что она что-то нашла.
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1

# Также мы должны указать порядок каналов во входном изображении. Обратите внимание, что это Blue Green Red (BGR), 
# а не привычный RGB. Это особенности работы данной модели.
cfg.INPUT.FORMAT = 'BGR' 

# Для более быстрой загрузки данных в модель, мы делаем параллельную загрузку. Мы указываем параметр 4, 
cfg.DATALOADER.NUM_WORKERS = 4

# Следующий параметр задает количество изображений в батче, на котором 
# модель делает одну итерацию обучения (изменения весов).
cfg.SOLVER.IMS_PER_BATCH = 1

# Зададим также learning_rate
cfg.SOLVER.BASE_LR = 0.01

# Укажем модели, через сколько шагов обучения модели следует уменьшить learning rate
cfg.SOLVER.STEPS = (1500,)

# Фактор, на который уменьшается learning rate задается следующим выражением
cfg.SOLVER.GAMMA = 0.1

# Зададим общее число итераций обучения.
cfg.SOLVER.MAX_ITER = 17000

# Укажем количество классов в нашей выборке
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

# Задаем через сколько  шагов обучения сохранять веса модели в файл. Этот файл мы сможем загрузить потом 
# для тестирования нашей обученной модели на новых данных.
cfg.SOLVER.CHECKPOINT_PERIOD = cfg.TEST.EVAL_PERIOD

cfg.TEST.DETECTIONS_PER_IMAGE = 1000

# cfg.MODEL.ROI_BOX_HEAD.TRAIN_ON_PRED_BOXES = True

# И указываем название папки, куда сохранять чекпойнты модели и информацию о процессе обучения.
cfg.OUTPUT_DIR = './output'

# Если вдруг такой папки нет, то создадим ее
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

# Если мы хотим удалить чекпойнты предыдущих моделей, то выполняем данную команду. 
#%rm output/*

In [ ]:
class custom_mapper:
    def __init__(self, cfg):
        self.transform_list = [
            T.ResizeShortestEdge(
                [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST],
                cfg.INPUT.MAX_SIZE_TEST),
            T.RandomBrightness(0.9, 1.1),
            T.RandomContrast(0.9, 1.1),
            T.RandomSaturation(0.9, 1.1),
            T.RandomLighting(0.9)
        ]
        print(f"[custom_mapper]: {self.transform_list}")

    def __call__(self, dataset_dict):
        dataset_dict = copy.deepcopy(dataset_dict)
        image = utils.read_image(dataset_dict["file_name"], format="BGR")
    
        image, transforms = T.apply_transform_gens(self.transform_list, image)
        dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

        annos = [
            utils.transform_instance_annotations(obj, transforms, image.shape[:2])
            for obj in dataset_dict.pop("annotations")
            if obj.get("iscrowd", 0) == 0
        ]

        instances = utils.annotations_to_instances(annos, image.shape[:2])
        dataset_dict["instances"] = utils.filter_empty_instances(instances)
        return dataset_dict

In [ ]:
def f1_loss(y_true, y_pred):
    tp = np.sum(y_true & y_pred)
    tn = np.sum(~y_true & ~y_pred)
    fp = np.sum(~y_true & y_pred)
    fn = np.sum(y_true & ~y_pred)
    
    epsilon = 1e-7
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    f1 = 2 * precision*recall / ( precision + recall + epsilon)
    return f1 

In [ ]:
CHECKPOINTS_RESULTS = []

class F1Evaluator(DatasetEvaluator):
    def __init__(self):
        self.loaded_true = np.load('data/train/binary.npz')
        self.val_predictions = {}
        self.f1_scores = []
        
    def reset(self):
        self.val_predictions = {}
        self.f1_scores = []

    def process(self, inputs, outputs):
        for input, output in zip(inputs, outputs):
            filename = input["file_name"].split("/")[-1]
            if filename != "41_3.JPG":
                true = self.loaded_true[filename].reshape(-1)

                prediction = output['instances'].pred_masks.cpu().numpy()
                mask = np.add.reduce(prediction)
                mask = (mask > 0).reshape(-1)

                self.f1_scores.append(f1_loss(true, mask))

    def evaluate(self):
        global CHECKPOINTS_RESULTS
        result = np.mean(self.f1_scores)
        CHECKPOINTS_RESULTS.append(result)
        return {"meanF1": result}

In [ ]:
class AugTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper(cfg))
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return F1Evaluator()

In [ ]:
class MyTrainer(DefaultTrainer):
    @classmethod
    def build_train_loader(cls, cfg, sampler=None):
        return build_detection_train_loader(
            cfg, mapper=MyMapper(cfg, True), sampler=sampler
        )
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        return F1Evaluator()

**4.2. Обучаем модель**

Процесс обучения модели запускают следующие три строчки кода. Возможно будут предупреждения, на которые можно не обращать внимания, это информация об обучении.

In [ ]:
%rm output/*

In [ ]:
trainer = MyTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()
print()

In [ ]:
del trainer
clear_cache()

In [ ]:
!ls ./output

Используем обученную модель для проверки качества на валидации.

In [ ]:
list(enumerate(CHECKPOINTS_RESULTS, start=1))

In [ ]:
with open("CHECKPOINTS_RESULTS.txt", "w") as f:
    f.write(str(CHECKPOINTS_RESULTS))